In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy import displacy
import json
import random
import re

In [2]:
#Import original data
df_orginal = pd.read_excel('./data/qb/original/ios.xlsx')

In [3]:
df_orginal.head()

,App,Store,App ID,Review ID,Country,Version,Rating,Date,Author,Subject,...,Translated Subject,Translated Body,Sentiment,Device,Language,OS Version,Reply URL,Topics,Custom Topics,Tags
0,QuickBooks Accounting,iOS,584606479,5.467988e+09,USA,20.01.6,1,2020-01-31,NanaHermanson,Shiny useless tool,...,NaN,NaN,negative,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Advertising"", ""Bugs"", ""Design & UX"", ""Dissat...","[""Ease of Use""]",[]
1,QuickBooks Accounting,iOS,584606479,5.464235e+09,Canada,20.01.5,1,2020-01-30,Tugboat55,Latest app update,...,NaN,NaN,negative,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Bugs"", ""Feature Requests"", ""Dissatisfied use...",[],[]
2,QuickBooks Accounting,iOS,584606479,5.467098e+09,USA,20.01.5,3,2020-01-30,scholarbev,App closes out while entering expenses,...,NaN,NaN,negative,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Bugs"", ""Frequency"", ""Pricing"", ""Customer Sup...","[""FDX Filter""]",[]
3,QuickBooks Accounting,iOS,584606479,5.466985e+09,USA,20.01.5,5,2020-01-30,SOSA EL DINERO,Life saver,...,NaN,NaN,positive,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Design & UX"", ""Performance"", ""Satisfied user...",[],[]
4,QuickBooks Accounting,iOS,584606479,5.467691e+09,USA,20.01.6,5,2020-01-30,Sra. Linda,Small business,...,NaN,NaN,positive,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Satisfied users""]",[],[]


In [4]:
#Get selective columns

df = df_orginal[['Review ID','Country', 'Version', 'Rating', 'Date', 'Subject', 'Body', 'Sentiment']]

In [5]:
df.head()

,Review ID,Country,Version,Rating,Date,Subject,Body,Sentiment
0,5.467988e+09,USA,20.01.6,1,2020-01-31,Shiny useless tool,This app looks awesome. Very well thought out...,negative
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,Latest app update,This update is terrible basically unusable ple...,negative
2,5.467098e+09,USA,20.01.5,3,2020-01-30,App closes out while entering expenses,The majority of the time this app is wonderful...,negative
3,5.466985e+09,USA,20.01.5,5,2020-01-30,Life saver,One my favorite apps to use while working. I’m...,positive
4,5.467691e+09,USA,20.01.6,5,2020-01-30,Small business,This absolutely works perfectly without diffic...,positive


In [6]:
df['Subject'] = df['Subject'].replace(np.nan, '', regex=True)
df['Body'] = df['Body'].replace(np.nan, '', regex=True)

/home/rmajumder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/rmajumder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df.head()

,Review ID,Country,Version,Rating,Date,Subject,Body,Sentiment
0,5.467988e+09,USA,20.01.6,1,2020-01-31,Shiny useless tool,This app looks awesome. Very well thought out...,negative
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,Latest app update,This update is terrible basically unusable ple...,negative
2,5.467098e+09,USA,20.01.5,3,2020-01-30,App closes out while entering expenses,The majority of the time this app is wonderful...,negative
3,5.466985e+09,USA,20.01.5,5,2020-01-30,Life saver,One my favorite apps to use while working. I’m...,positive
4,5.467691e+09,USA,20.01.6,5,2020-01-30,Small business,This absolutely works perfectly without diffic...,positive


In [8]:
#combine subject and body
df['Review'] = df[['Subject', 'Body']].apply(lambda x: '. '.join(x), axis=1)

/home/rmajumder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df.head()

,Review ID,Country,Version,Rating,Date,Subject,Body,Sentiment,Review
0,5.467988e+09,USA,20.01.6,1,2020-01-31,Shiny useless tool,This app looks awesome. Very well thought out...,negative,Shiny useless tool. This app looks awesome. V...
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,Latest app update,This update is terrible basically unusable ple...,negative,Latest app update. This update is terrible bas...
2,5.467098e+09,USA,20.01.5,3,2020-01-30,App closes out while entering expenses,The majority of the time this app is wonderful...,negative,App closes out while entering expenses. The ma...
3,5.466985e+09,USA,20.01.5,5,2020-01-30,Life saver,One my favorite apps to use while working. I’m...,positive,Life saver. One my favorite apps to use while ...
4,5.467691e+09,USA,20.01.6,5,2020-01-30,Small business,This absolutely works perfectly without diffic...,positive,Small business. This absolutely works perfectl...


In [10]:
#Drop Subject and Body

del df['Subject']
del df['Body']

In [11]:
df.head()

,Review ID,Country,Version,Rating,Date,Sentiment,Review
0,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,Shiny useless tool. This app looks awesome. V...
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,Latest app update. This update is terrible bas...
2,5.467098e+09,USA,20.01.5,3,2020-01-30,negative,App closes out while entering expenses. The ma...
3,5.466985e+09,USA,20.01.5,5,2020-01-30,positive,Life saver. One my favorite apps to use while ...
4,5.467691e+09,USA,20.01.6,5,2020-01-30,positive,Small business. This absolutely works perfectl...


In [12]:
filename = "./data/qb/processed/train_ner.json"
print(filename)


with open(filename) as train_data:
    train = json.load(train_data)

TRAIN_DATA_NEW = []
for data in train:
    ents = [tuple(entity) for entity in data['entities']]
    TRAIN_DATA_NEW.append((data['content'],{'entities':ents}))


with open('{}'.format(filename.replace('json','txt')),'w') as write:
    write.write(str(TRAIN_DATA_NEW))


./data/qb/processed/train_ner.json


In [13]:

def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.load('en_core_web_sm')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA_NEW, 20)

Statring iteration 0
{'ner': 1901.8735729833259}
Statring iteration 1
{'ner': 2035.789364135086}
Statring iteration 2
{'ner': 2044.2571074728507}
Statring iteration 3
{'ner': 2060.4287681180085}
Statring iteration 4
{'ner': 2052.722274809328}
Statring iteration 5
{'ner': 2040.9560997235674}
Statring iteration 6
{'ner': 1946.0919011448418}
Statring iteration 7
{'ner': 1905.1645837245287}
Statring iteration 8
{'ner': 1824.5535311963065}
Statring iteration 9
{'ner': 1956.088040700828}
Statring iteration 10
{'ner': 1941.335229467149}
Statring iteration 11
{'ner': 1799.5049142112737}
Statring iteration 12
{'ner': 1842.7346244760042}
Statring iteration 13
{'ner': 1918.741944829124}
Statring iteration 14
{'ner': 1752.3588071444758}
Statring iteration 15
{'ner': 1824.7223118712031}
Statring iteration 16
{'ner': 1806.749991743212}
Statring iteration 17
{'ner': 1901.8086431990755}
Statring iteration 18
{'ner': 1787.9113918621056}
Statring iteration 19
{'ner': 1850.2017864376996}


In [15]:
# Save our trained Model
prdnlp.to_disk("./model/qbo_aspect")

In [16]:
#Test your text
test_text = ["Cancelling your subscription seems impossible. I've emailed, called, used their help menu. It shouldn't be this hard. Still trying to cancel.",
              "My App has stopped working on my Samsung s9 for 2 days in a row. You should have sent an email to inform subscribers if your app is going to be experiencing a downtime.",
              "Cannot open customer invoices. Please correct ASAP!",
               "bugs on iOS13 please update ASAP!"]
for text in test_text:
    print('-----')
    doc = prdnlp(text)
    for ent in doc.ents:
        print('keyword=' + ent.text, 'aspect=' + ent.label_)

-----
keyword=subscription aspect=Subscription
keyword=called aspect=Customer Service
-----
keyword=App aspect=App
keyword=app aspect=App
-----
keyword=invoices aspect=Invoicing
-----
keyword=bugs aspect=Quality


In [17]:
for index, row in df.iterrows():
    aspect_terms = []
    aspect_related_text = []
    doc = prdnlp(row['Review'])
    for ent in doc.ents:
        if ent.label_ not in aspect_terms:            
            aspect_terms.append(ent.label_)
            aspect_related_text.append(str(ent))

    df.loc[index,'Aspects'] = ", ".join(aspect_terms)
    df.loc[index,'AspectText'] = ", ".join(aspect_related_text)

/home/rmajumder/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/rmajumder/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [18]:
df.head()

,Review ID,Country,Version,Rating,Date,Sentiment,Review,Aspects,AspectText
0,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,Shiny useless tool. This app looks awesome. V...,"App, Expenses, Help, Customer Service","app, expenses, videos, support"
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,Latest app update. This update is terrible bas...,App,app
2,5.467098e+09,USA,20.01.5,3,2020-01-30,negative,App closes out while entering expenses. The ma...,"App, Transactions, Subscription","App, transaction, subscription"
3,5.466985e+09,USA,20.01.5,5,2020-01-30,positive,Life saver. One my favorite apps to use while ...,App,apps
4,5.467691e+09,USA,20.01.6,5,2020-01-30,positive,Small business. This absolutely works perfectl...,,


In [19]:
#number of reviews without aspect
(df['Aspects'].values == '').sum() 

1827

In [20]:
len(df)

7560

In [21]:
df['Aspects'].replace('', np.nan, inplace=True)
df.dropna(subset=['Aspects'], inplace=True)

/home/rmajumder/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/rmajumder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
len(df)

5733

In [57]:
df.head(50)

,Review ID,Country,Version,Rating,Date,Sentiment,Review,Aspects,AspectText
0,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,Shiny useless tool. This app looks awesome. V...,"App, Expenses, Help, Customer Service","app, expenses, videos, support"
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,Latest app update. This update is terrible bas...,App,app
2,5.467098e+09,USA,20.01.5,3,2020-01-30,negative,App closes out while entering expenses. The ma...,"App, Transactions, Subscription","App, transaction, subscription"
3,5.466985e+09,USA,20.01.5,5,2020-01-30,positive,Life saver. One my favorite apps to use while ...,App,apps
5,5.460553e+09,USA,20.01.4,1,2020-01-29,negative,Worst update yet. Just deleted. Crashed. Didn’...,Quality,deleted
6,5.461476e+09,USA,20.01.4,1,2020-01-29,negative,Transferring to TurboTax. Transferring company...,Quality,TurboTax
7,5.462530e+09,USA,20.01.4,5,2020-01-29,positive,Great app for small business owners!. I have a...,App,app
8,5.462811e+09,USA,20.01.4,4,2020-01-29,mixed,Good but needs multi currency. Very easy to us...,Invoicing,invoice
9,5.463636e+09,USA,20.01.4,5,2020-01-29,neutral,"CEO, Bear Island Brewing Co. I was hesitant to...",Quality,Co.
10,5.457349e+09,United Kingdom,20.01.4,4,2020-01-28,neutral,Good App but. It’s a good app but you should b...,"App, Invoicing","App, invoices"


In [71]:
#generate aspect position index

def generate_df_with_position(data_frame):

    df_split_terms = pd.DataFrame(columns=['Review ID', 
                                           'Country', 'Version', 
                                           'Rating', 'Date', 'Doc Sentiment', 'Asp Sentiment', 
                                           'Review', 'Aspects', 'AspectText', 'Positions'])


    for index, row in data_frame.iterrows():
        aspecttexts = [x.strip() for x in row['AspectText'].lower().split(',')]
        aspects = [x.strip() for x in row['Aspects'].lower().split(',')]

        positions = ''
        for idx, asp in enumerate(aspecttexts):

            review = " ".join(row['Review'].lower().split())

            if len(review.split(' ')) > 85:
                continue

            start_idx = review.find(asp)

            #if the aspect is not found in the review, skip
            if start_idx == -1:
                continue

            sentiment = 'neutral' if row['Sentiment'] == 'mixed' else row['Sentiment']


            df_split_terms = df_split_terms.append({
                'Review ID': row['Review ID'], 
                'Country': row['Country'], 
                'Version': row['Version'],
                'Rating': row['Rating'],
                'Date': row['Date'],            
                'Doc Sentiment': sentiment,
                'Asp Sentiment' : '' if len(aspecttexts) > 1 or sentiment == 'neutral' else sentiment,
                'Review': review,
                'AspectText': asp,
                'Aspects': aspects[idx] if len(aspects) > idx else asp,
                'Positions': str(start_idx) + ',' + str(start_idx + len(asp) - 1),
            }, ignore_index=True)

    return df_split_terms
            

In [72]:
df_split_terms = generate_df_with_position(df)
df_split_terms.head()

,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
0,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,negative,latest app update. this update is terrible bas...,app,app,"7,9"
1,5.466985e+09,USA,20.01.5,5,2020-01-30,positive,positive,life saver. one my favorite apps to use while ...,app,apps,"28,31"
2,5.460553e+09,USA,20.01.4,1,2020-01-29,negative,negative,worst update yet. just deleted. crashed. didn’...,quality,deleted,"23,29"
3,5.461476e+09,USA,20.01.4,1,2020-01-29,negative,negative,transferring to turbotax. transferring company...,quality,turbotax,"16,23"
4,5.462530e+09,USA,20.01.4,5,2020-01-29,positive,positive,great app for small business owners!. i have a...,app,app,"6,8"


In [73]:
len(df_split_terms)

7004

In [54]:

df_split_terms.to_csv(r'data/qb/processed/ios_aspect_position.csv')

In [116]:
#Reimport after aspect/position/sentiment refinements
df_position_sentiment = pd.read_csv(r'./data/qb/processed/IOS Aspect - Position and Sentiment.csv')

In [117]:
df_position_sentiment.head()

,Unnamed: 0,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
0,0,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,negative,latest app update. this update is terrible bas...,app,app,"7,9"
1,1,5.466985e+09,USA,20.01.5,5,2020-01-30,positive,positive,life saver. one my favorite apps to use while ...,app,apps,"28,31"
2,2,5.460553e+09,USA,20.01.4,1,2020-01-29,negative,negative,worst update yet. just deleted. crashed. didn’...,quality,deleted,"23,29"
3,3,5.461476e+09,USA,20.01.4,1,2020-01-29,negative,negative,transferring to turbotax. transferring company...,quality,turbotax,"16,23"
4,4,5.462530e+09,USA,20.01.4,5,2020-01-29,positive,positive,great app for small business owners!. i have a...,app,app,"6,8"


In [118]:
#df_position_sentiment[df_position_sentiment.AspectText == ]
print(len(df_position_sentiment))
df_position_sentiment.dropna(subset=['AspectText'], inplace=True)
print(len(df_position_sentiment))

7003
7000


In [130]:
#recalculate position
for index, row in df_position_sentiment.iterrows():
    
    if len(row['Review']) > 85:
        print(row['Review'])
    
    df_position_sentiment.loc[index,'Review'] = row['Review'].replace('/', '').replace('-', '')
    
    start_idx = row['Review'].find(row['AspectText'])
          
    df_position_sentiment.loc[index,'Positions'] = str(start_idx) + ',' + str(start_idx + len(asp) - 1)
    

,Unnamed: 0,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions


In [131]:
df_position_sentiment.head()

,Unnamed: 0,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
0,0,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,negative,latest app update. this update is terrible bas...,app,app,"7,9"
1,1,5.466985e+09,USA,20.01.5,5,2020-01-30,positive,positive,life saver. one my favorite apps to use while ...,app,apps,"28,30"
2,2,5.460553e+09,USA,20.01.4,1,2020-01-29,negative,negative,worst update yet. just deleted. crashed. didn’...,quality,deleted,"23,25"
3,3,5.461476e+09,USA,20.01.4,1,2020-01-29,negative,negative,transferring to turbotax. transferring company...,quality,turbotax,"16,18"
4,4,5.462530e+09,USA,20.01.4,5,2020-01-29,positive,positive,great app for small business owners!. i have a...,app,app,"6,8"


In [132]:
#split data in - train test and dev
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    
    perm = np.random.permutation(np.arange(len(df)))
    m = len(df)
    
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end - 1] - 1]
    validate = df.iloc[perm[train_end:validate_end - 1] - 1]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [133]:
train, validate, test = train_validate_test_split(df_position_sentiment)

In [134]:
train.head()

,Unnamed: 0,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
5737,5738,1.268408e+09,USA,4.3.2,4,2015-10-07,positive,positive,"love the convenience. love the app, but please...",app,app,"31,33"
1500,1500,2.604446e+09,USA,18.4,1,2018-05-27,negative,negative,doesn’t work like the app on a pc. the fact th...,experience,ipad,"95,97"
1859,1859,1.945504e+09,USA,17.11,1,2017-11-24,negative,negative,bank connections never work. bank connections ...,quality,transactions,"49,51"
4576,4576,1.430178e+09,Mexico,4.5.1,5,2016-08-09,positive,positive,great app.,app,app,"6,8"
3570,3570,1.513958e+09,USA,5.0.3,5,2017-01-02,positive,positive,1st time user. loving it so far,app,user,"9,11"


In [135]:
len(train)

4199

### Generate files

In [136]:
def generate_input_files(d_type, data):

    #save reviews
    df_review = data[['Review']]
    df_review.to_csv(r'data/qb/{}/review.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save aspects
    df_review = data[['AspectText']]
    df_review.to_csv(r'data/qb/{}/term.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save positions
    df_review = data[['Positions']]
    df_review.to_csv(r'data/qb/{}/position.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save label
    df_review = data[['Asp Sentiment']]
    df_review.to_csv(r'data/qb/{}/label.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

In [137]:
generate_input_files('train', train)
generate_input_files('test', test)
generate_input_files('dev', validate)